In [15]:
import plotly.express as px
import pandas as pd
import json
import datetime
from pytimeparse.timeparse import timeparse
from datetime import timedelta

# Load rostering data
with open(f'./rostering.json', 'r', encoding='utf-8') as f:
        rostering = json.load(f)

# Load rostering data
with open(f'./scheduling_meta_input.json', 'r', encoding='utf-8') as f:
        meta = json.load(f)

shifts_duration = {}
for s in meta['shifts']:
    shifts_duration[s['id']] = datetime.timedelta(minutes= int(s['duration'][:-3]) * 60+ int(s['duration'][-2:]))

print(shifts_duration)

{'9часов день': datetime.timedelta(seconds=32400), '9часов ночь': datetime.timedelta(seconds=32400), '12часов день': datetime.timedelta(seconds=43200), '12часов ночь': datetime.timedelta(seconds=43200)}


In [27]:


format = '%d.%m.%y %H:%M'
shifts = map(
        lambda x: dict(Task = str(x['employeeId']),
                       Start = datetime.datetime.strptime(f"{x['shiftDate']} {x['shiftTimeStart']}", format),
                       Finish = datetime.datetime.strptime(f"{x['shiftDate']} {x['shiftTimeStart']}", format) + shifts_duration[x['shiftId']],
                       Resource = x['shiftId'])
        , rostering['campainSchedule']
)

df = pd.DataFrame(shifts)
df.head()

,Task,Start,Finish,Resource
0,6807,2022-12-05 09:15:00,2022-12-05 21:15:00,12часов день
1,6807,2022-12-06 06:00:00,2022-12-06 18:00:00,12часов день
2,6807,2022-12-07 09:15:00,2022-12-07 21:15:00,12часов день
3,6807,2022-12-08 10:45:00,2022-12-08 22:45:00,12часов день
4,6807,2022-12-09 06:00:00,2022-12-09 18:00:00,12часов день


In [28]:
fig = px.timeline(df, x_start="Start", x_end="Finish", y="Task", color="Resource", height = 10000)
fig.update_yaxes(autorange="reversed")
fig.update_xaxes(rangeslider_visible=True)
fig.show()
